In [1]:
# Atnaujintas variantas, po to, kaip padaryti pakeitimai failų varduose 2022 03 26
# reikia dar tvarkyti ///////////////////////////////////////////////////////////
# Nuskaito ir parodo anotacijas visiems failams iš sąrašo. Sąrašas sukuriamas su 0_zive_suppl_list

import pandas as pd
import numpy as np

import json, sys
from pathlib import Path
from icecream import ic

def get_annotations_distribution(df_list, rec_dir, beats_annot, flag_incl):
# Jei flag_incl == False, į požymį incl nereaguoti, sąrašą išvesti visiems EKG įrašams.
# Jei flag_incl == True, sąrašą išvęsti tik tiems įrašams, kuriems incl ==1

    # Nustatomas anotacijų pasiskirstymas per visus įrašus.
    # Pasiruošimas ciklui per pacientų įrašus
    labels_rec_all = pd.DataFrame(columns=beats_annot.keys(),dtype=int)
    labels_rec_all.insert(0,"file_name",0)
    labels_rec_all = labels_rec_all.astype({"file_name":str})
    labels_rec_all.insert(1,"userId",0)
    labels_rec_all = labels_rec_all.astype({"userId":str})
    labels_rec_all.insert(2,"recordingId",0)
    labels_rec_all = labels_rec_all.astype({"recordingId":str})
    labels_rec_all.insert(8,"incl",0)
    labels_rec_all = labels_rec_all.astype({"incl":int})
    labels_rec_all.insert(9,"recorded_at",0)
    labels_rec_all = labels_rec_all.astype({"recorded_at":str})

    labels_rec = []

    # Ciklas per pacientų įrašus
    for ind in df_list.index:
        file_name = df_list.loc[ind, "file_name"]
        labels_rec = np.zeros(labels_rec_all.shape[1], dtype=int)
        recordingId = df_list.loc[ind, "recordingId"]
        file_name = df_list.loc[ind, "file_name"]


        #  load data using Python JSON module
        file_path = Path(rec_dir, file_name + '.json')
        with open(file_path,'r', encoding='UTF-8', errors = 'ignore') as f:
            data = json.loads(f.read())

        df_rpeaks = pd.json_normalize(data, record_path =['rpeaks'])

        atr_sample = df_rpeaks['sampleIndex'].to_numpy()
        atr_symbol = df_rpeaks['annotationValue'].to_numpy()

        # Ciklas per visas paciento įrašo anotacijas (simbolius)
        for symbol in atr_symbol:
            # Gaunamas anotacijos simbolio numeris anotacijų sąraše
            label = beats_annot.get(symbol)
            if (label == None):
                continue
            labels_rec[label] +=1

        # timestamp_str = time.strftime(  '%m/%d/%Y :: %H:%M:%S', time.gmtime(os.path.getmtime(file_path))) 
        # timestamp_str = time.strftime(  '%m/%d/%Y :: %H:%M:%S', time.gmtime(os.path.getmtime(file_path))) 
        recorded_at = df_list.loc[ind, "recorded_at"]
        # recorded_at = datetime.fromtimestamp(timestamp, tz=timezone.utc)

        # Sumuojame į bendrą masyvą
        dict = {'file_name':file_name, 'userId': data['userId'], 'recordingId':recordingId,  'N':labels_rec[0], 'S':labels_rec[1],
         'V':labels_rec[2], 'F':labels_rec[3], 'U':labels_rec[4], 'incl':df_list.loc[ind, "incl"], 'recorded_at':recorded_at}
        if (flag_incl == True):
            if (df_list.loc[ind, "incl"] == 1): 
                labels_rec_all = labels_rec_all.append(dict, ignore_index = True)
        else:
            labels_rec_all = labels_rec_all.append(dict, ignore_index = True)

    # Ciklo per pacientų įrašus pabaiga
    return labels_rec_all

import warnings
warnings.filterwarnings("ignore")

my_os=sys.platform
print("OS in my system : ", my_os)

if my_os != 'linux':
    OS = 'Windows'
else:  
    OS = 'Ubuntu'

# Pasiruošimas

# //////////////// NURODOMI PARAMETRAI /////////////////////////////////////////////////////

# Bendras duomenų aplankas

if OS == 'Windows':
    Duomenu_aplankas = 'D:\DI\Data\MIT&ZIVE'   # variantas: Windows
else:
    Duomenu_aplankas = '/home/kesju/DI/Data/MIT&ZIVE'   # arba variantas: UBUNTU, be Docker

# jei variantas Docker pasirenkame:
# Duomenu_aplankas = '/Data/MIT&ZIVE'

# Vietinės talpyklos aplankas
db_folder = 'ZIVE_BUFFER_2022_03_24'

# Aplankas su  EKG įrašais ir anotacijomis (.json)
rec_folder = 'records'

# ///////////////////////////////////////////////////////////////////////////////////////////
print('\nSkriptas parodo anotacijų statistiką vietinėje ZIVE talpykloje')

pd.set_option("display.max_rows", 6000, "display.max_columns", 20)
pd.set_option('display.width', 1000)
# How to print an entire pandas DataFrame in Python
# https://www.kite.com/python/answers/how-to-print-an-entire-pandas-dataframe-in-python

# Nuoroda į aplanką su vietiniu EKG duomenų rinkiniu
db_path = Path(Duomenu_aplankas, db_folder, rec_folder) # vietinė talpykla su jau esamais duomenimis arba tuščia
print( 'Vietinės talpyklos aplankas:', db_folder)

# Nuskaitomas įrašų sąrašas, suformuojamas atitinkamas dataframe df_list
file_path = Path(db_path, 'list.json')
with open(file_path,'r', encoding='UTF-8', errors = 'ignore') as f:
    data = json.loads(f.read())
df_list = pd.json_normalize(data, record_path =['data'])

# Susirandame anotacijų pasiskirstymą per įrašus
annot_list = ['N', 'S', 'V', 'F', 'U']
# Susikuriame pagalbinį anotacijų žodyną - dictionary beats_annot
nr_sequence = list(range(len(annot_list)))
beats_annot = dict(zip(annot_list, nr_sequence))
labels_rec_all = get_annotations_distribution(df_list, db_path, beats_annot, False)

# Susirandame anotacijų pasiskirstymą per pacientus ir pacientų skaičių
df_sum = labels_rec_all.groupby(['userId'],sort = False).sum()
# https://sparkbyexamples.com/pandas/pandas-groupby-sum-examples/
count = labels_rec_all['userId'].value_counts()
print(f'\nViso pacientų: {len(count)}  EKG įrašų: {len(labels_rec_all)}')
print(f'\nĮrašų pasiskirstymas per pacientus')
count = count.rename("count")
frames = [df_sum, count]
result = pd.concat(frames, axis = 1)
result.index.rename ('userID', inplace= True)
result.drop(labels=['incl'], axis=1, inplace=True)
# https://www.shanelynn.ie/pandas-drop-delete-dataframe-rows-columns/
print(result)

filepath = Path(db_path, 'result.csv') 
result.to_csv(filepath)
print(f'Failų sąrašas ir rezultatai įrašyti į:  {filepath}')    

# Susumuojamos anotacijos
suma = labels_rec_all.iloc[:,3:8].sum()
print('\n',' '*18, 'sum: ',str(suma.tolist())[1:-1])
total = suma.sum()
print(' '*17, 'total: ', total)

# Parodomas anotacijų ir užduotų klasių pasiskirstymas per visus įrašus
print("\nAnotacijų pasiskirstymas per visus įrašus\n")
print(labels_rec_all)
suma = labels_rec_all.iloc[:,3:8].sum()
print('\n',' '*51, 'sum: ',str(suma.tolist())[1:-1])
total = suma.sum()
print(' '*50, 'total: ', total)


grouped  = labels_rec_all.groupby('userId')
for userId, group in grouped:
    # print(group.dtypes)
    print("\n")
    # userId = get_rec_userId(rec_dir, userNr)
    # print(f"{'userNr:'} {userNr} {'userId:'} {userId}" )
    print(f"{'userId:'} {userId}" )
    # tit1 = f"{'userNr':>6} {'recNr':>6}  {'recId':>25} {'N':>8} {'S':>4} {'V':>4}"
    # tit2 = f"{'Nprec':>8} {'Nrec':>5} {'Nfsc':>5}"
    # tit3 = f"{'Sprec':>8} {'Srec':>5} {'Sfsc':>5}"
    # tit4 = f"{'Vprec':>8} {'Vrec':>5} {'Vfsc':>5} {'Err%':>8} {'Noise%':>8}"
    # print(tit1+tit2+tit3+tit4)

    for idx, row in group.iterrows():
        print(f"{'file_name:'} {str(row['file_name'])}" )
        # str1 =f"{int(row['userNr']):>6} {int(row['recNr']):>6} {str(row['recId']):>26} {int(row['N']):>8} {int(row['S']):4} {int(row['V']):4}" 
        # str2 = f"{row['Nprec']:>8.2f} {row['Nrec']:5.2f} {row['Nfsc']:5.2f}"
        # str3 = f"{row['Sprec']:>8.2f} {row['Srec']:5.2f} {row['Sfsc']:5.2f}"
        # str4 = f"{row['Vprec']:>8.2f} {row['Vrec']:5.2f} {row['Vfsc']:5.2f} {row['Err%']:8.1f} {row['Noise%']:8.1f}"
        # print(str1+str2+str3+str4)

OS in my system :  linux

Skriptas parodo anotacijų statistiką vietinėje ZIVE talpykloje
Vietinės talpyklos aplankas: ZIVE_BUFFER_2022_03_24

Viso pacientų: 4  EKG įrašų: 4

Įrašų pasiskirstymas per pacientus
                            N   S    V  F  U  count
userID                                             
6144c5b1bd0cc5a681275363  903   0    0  0  0      1
6144c4fbbd0cc552e427535f  541   0  194  0  0      1
613b1d673d08d4d1f3cdc8f8  656  12    0  0  0      1
6190d4b23cd1d29db2303ce9  600  11    2  0  0      1
Failų sąrašas ir rezultatai įrašyti į:  /home/kesju/DI/Data/MIT&ZIVE/ZIVE_BUFFER_2022_03_24/records/result.csv

                    sum:  2700, 23, 196, 0, 0
                  total:  2919

Anotacijų pasiskirstymas per visus įrašus

     file_name                    userId               recordingId    N   S    V  F  U  incl               recorded_at
0  1630844.567  6144c5b1bd0cc5a681275363  6145aeb6bd0cc5e8392757d1  903   0    0  0  0     0  2021-09-05T12:22:47.000Z
1  16310